In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
from collections import defaultdict
import nltk
from gensim.models import Word2Vec
import random
import torch.nn.functional as F
import numpy as np

# TODO
Need to figure out how to parse data to have the review + the label<br>
Need to figure out how to build the baseline neural network<br>
Need to figure out how 

In [2]:
data = defaultdict(list)
with open("../review.json", "r") as file:
    for line in file:
        d = json.loads(line)
        data[d['funny'] > 0].append((d['review_id'], d['text'], d['funny'] > 0))

In [3]:
num_samples = 60000

In [4]:
# Sample the data
sentences = random.sample(data[True], num_samples)
sentences.extend(random.sample(data[False], num_samples))
random.shuffle(sentences)

del data[True]
del data[False]

In [5]:
# Parse the sentences
def parseSentenceBasic(sentence):
    sent = sentence[1].strip().split()
    return [x.lower() for x in sent]

def parseSentences(sentences, p_func):
    corpus = []
    for s in sentences:
        corpus.append(p_func(s))
    return corpus

In [6]:
# Create the word embedding model based on the current corpus
corpus = parseSentences(sentences, parseSentenceBasic)
word2Vec = Word2Vec(corpus, size=256, min_count=1)

In [7]:
def getWordEmbedding(word, word2Vec):
    try:
        return torch.from_numpy(word2Vec.wv[word])
    except KeyError:
        return torch.from_numpy(word2Vec.wv["<UNK>"])
    
def getWordEmbedding2(word, word2Vec):
    try:
        return word2Vec.wv[word]
    except KeyError:
        return word2Vec.wv["<UNK>"]

# Baseline Network
TODO: Figure out what you actually need to do lol<br>
So far, I have the network outlined, but we need to figure out how to train the data<br>
We also need to figure out how to define a loss function

In [8]:
device = torch.device("cuda:0")

In [ ]:
class ShallowNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(256, 100)
        self.hidden2 = nn.Linear(100, 150)
        self.output = nn.Linear(150, 2)
    
    def forward(self, X):
        X.to(device)
        
        q1 = self.hidden1(X)
        h1 = F.relu(q1)
        q2 = self.hidden2(h1)
        h2 = F.relu(q2)
        
        return self.output(h2)

In [23]:
shallow_net

ShallowNetwork(
  (hidden1): Linear(in_features=256, out_features=100, bias=True)
  (hidden2): Linear(in_features=100, out_features=150, bias=True)
  (output): Linear(in_features=150, out_features=2, bias=True)
)

In [11]:
torch.cuda.is_available()

True

In [12]:
# Turn the corpus into a list of vectors for input
def reviewToVecNaive(s):
    result = torch.zeros(len(s),256)
    for i in range(len(s)):
        result[i] = getWordEmbedding(s[i], word2Vec)
    return torch.sum(result, 0)

def reviewToVecNaive2(s):
    result = torch.zeros(256)
    for i in range(len(s)):
        result = result.add(getWordEmbedding(s[i], word2Vec))
    return result

def reviewToVecNaive3(s):
    result = np.zeros((len(s), 256), dtype=np.float32)
    for i in range(len(s)):
        result[i] = getWordEmbedding(s[i], word2Vec)
    return torch.from_numpy(np.sum(result, axis=0))

In [13]:
N = len(corpus)
corpusVecs1 = [reviewToVecNaive3(s) for s in corpus[:N]]

In [45]:
trainIdx = 20000

In [72]:
from torch import optim
from torch.autograd import Variable

shallow_net = ShallowNetwork()
shallow_net.cuda()
loss_function = nn.CrossEntropyLoss()

def printOptInfo(optimizer):
    print(optimizer.__class__.__name__, optimizer.param_groups[0]['lr'])

def trainShallowNet(optimizer, epochs, corpusVecs, printResults=True):
    if printResults:
        printOptInfo(optimizer)
    for epoch in range(epochs): 
        train_loss = []
        shallow_net.train()
        total_loss = total_correct = 0
        total = trainIdx

        for i in range(trainIdx):
            optimizer.zero_grad()

            corpusVecs[i] = corpusVecs[i].to(device)

            output = shallow_net(corpusVecs[i])
            output = torch.unsqueeze(output, 0)
            output = output.to(device)

            _, x = output.data.topk(1)
            target = torch.tensor([int(sentences[i][2])])
            target = target.to(device)

            if x[0][0] == target:
                total_correct += 1

            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        if printResults:
            print("epoch: {}, loss: {}, accuracy: {}".format(epoch, np.mean(train_loss), total_correct/total))

In [63]:
def printOptInfo(optimizer):
    print(optimizer.__class__.__name__, optimizer.param_groups[0]['lr'])

In [66]:
# Testing results for SDG optimizer with constant lr, diff epochs
shallow_net = ShallowNetwork().cuda()
o1 = optim.SGD(shallow_net.parameters(), lr=0.001)
trainShallowNet(o1, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o2 = optim.SGD(shallow_net.parameters(), lr=0.001)
trainShallowNet(o2, 8, corpusVecs1)

SGD 0.001
epoch: 0, loss: 0.6765921560894698, accuracy: 0.5942
epoch: 1, loss: 0.6789048265516758, accuracy: 0.57205
SGD 0.001
epoch: 0, loss: 0.6757887674849481, accuracy: 0.58495
epoch: 1, loss: 0.6723869769826532, accuracy: 0.592
epoch: 2, loss: 0.6770411373376847, accuracy: 0.5788
epoch: 3, loss: 0.677973353369534, accuracy: 0.57975
epoch: 4, loss: 0.6783318033911288, accuracy: 0.5763
epoch: 5, loss: 0.680336127974838, accuracy: 0.57725
epoch: 6, loss: 0.679368626422435, accuracy: 0.58505
epoch: 7, loss: 0.6716068382002414, accuracy: 0.6017


In [76]:
shallow_net.zero_grad()

In [79]:
# Testing results for SDG optimizer with constant epochs, diff lr
shallow_net = ShallowNetwork().cuda()
o3 = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o3, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o4 = optim.SGD(shallow_net.parameters(), lr=0.01)
trainShallowNet(o4, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o5 = optim.SGD(shallow_net.parameters(), lr=0.1)
trainShallowNet(o5, 2, corpusVecs1)

SGD 0.0001
epoch: 0, loss: 0.697416302045621, accuracy: 0.60095
epoch: 1, loss: 0.6538178123567253, accuracy: 0.62615
SGD 0.01
epoch: 0, loss: nan, accuracy: 0.0195
epoch: 1, loss: nan, accuracy: 0.0
SGD 0.1
epoch: 0, loss: nan, accuracy: 0.00335
epoch: 1, loss: nan, accuracy: 0.0


In [81]:
shallow_net = ShallowNetwork().cuda()
o_best_sgd = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o_best_sgd, 10, corpusVecs1)

SGD 0.0001
epoch: 0, loss: 0.7003838335053064, accuracy: 0.5994
epoch: 1, loss: 0.6560078274384141, accuracy: 0.62375
epoch: 2, loss: 0.6496948119603098, accuracy: 0.6324
epoch: 3, loss: 0.6455893912747502, accuracy: 0.6383
epoch: 4, loss: 0.6431354025945067, accuracy: 0.6404
epoch: 5, loss: 0.6400601583058014, accuracy: 0.64185
epoch: 6, loss: 0.6372200701104477, accuracy: 0.64595
epoch: 7, loss: 0.6354311815664172, accuracy: 0.6462
epoch: 8, loss: 0.633373790150322, accuracy: 0.64905
epoch: 9, loss: 0.6313983280704357, accuracy: 0.6485


In [80]:
# Testing results with Adam with 2 epochs
shallow_net = ShallowNetwork().cuda()
o6 = optim.Adam(shallow_net.parameters(), lr=0.1, betas=(0.9, 0.99)) # Starting with the best results found in the paper
trainShallowNet(o6, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o7 = optim.Adam(shallow_net.parameters(), lr=0.01) # Starting with the best results found in the paper
trainShallowNet(o7, 2, corpusVecs1)

Adam 0.1
epoch: 0, loss: 606.7561558526859, accuracy: 0.49975
epoch: 1, loss: 4.72142732155174, accuracy: 0.4976
Adam 0.01
epoch: 0, loss: 3.656859737342596, accuracy: 0.499
epoch: 1, loss: 0.7507467496782542, accuracy: 0.4952


In [82]:
# Testing results with Adam with more epochs using best parameters from before
shallow_net = ShallowNetwork().cuda()
o_best_adam = optim.Adam(shallow_net.parameters(), lr=0.1, betas=(0.9, 0.99)) # Starting with the best results found in the paper
trainShallowNet(o_best_adam, 8, corpusVecs1)

Adam 0.1
epoch: 0, loss: 325.3814041033566, accuracy: 0.4988
epoch: 1, loss: 0.7207487521916628, accuracy: 0.49715
epoch: 2, loss: 0.7207487520843744, accuracy: 0.49715
epoch: 3, loss: 0.7207487520709633, accuracy: 0.49715
epoch: 4, loss: 0.7207487522661686, accuracy: 0.49715
epoch: 5, loss: 0.7207487523317337, accuracy: 0.49715
epoch: 6, loss: 0.7207487522795797, accuracy: 0.49715
epoch: 7, loss: 0.7207487520948053, accuracy: 0.49715


# TODO: Run this overnight b/c it's a lot of epochs lmao

In [ ]:
shallow_net = ShallowNetwork().cuda()
o_best_sgd = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o_best_sgd, 50, corpusVecs1)

## After playing around with the training set, apply results to large test set

In [85]:
# Test accuracy
testIdx = len(corpusVecs1)

def testShallowNet(optimizer, epochs, corpusVecs, printResults=True):
    shallow_net.eval()
    if printResults:
        printOptInfo(optimizer)
        
    valid_loss = []
    vtc = 0
    total = testIdx - trainIdx

    for i in range(trainIdx, testIdx):
        corpusVecs[i] = corpusVecs[i].to(device)

        output = shallow_net(corpusVecs1[i])
        output = torch.unsqueeze(output,0)
        output = output.to(device)

        _, x = output.data.topk(1)
        target = torch.tensor([int(sentences[i][2])])
        target = target.to(device)

        if x[0][0] == target:
            vtc += 1

        loss = loss_function(output, target)
        valid_loss.append(loss.item())

    if printResults:
        print ("Valid Loss: ", np.mean(valid_loss), " Valid Accuracy: ", vtc/total)

# TODO: Run this overnight too to get the test data

In [ ]:
shallow_net = ShallowNetwork().cuda()
best_optim = 0 # REPLACE
best_epochs = 0 # REPLACE
testShallowNet(best_optim, best_epochs, corpusVecs1)

# Build other datasets with diff embeddings using best optimizer

In [98]:
### Remove Punctuation ###
import string
def parseSentenceRemovePunc(sentence):
    sent = sentence[1]
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    sent = sent.strip().split()
    return [x.lower() for x in sent]

def reviewToVecRemovePunc(s):
    result = np.zeros((len(s), 256), dtype=np.float32)
    for i in range(len(s)):
        result[i] = getWordEmbedding(s[i], word2VecRP)
    return torch.from_numpy(np.sum(result, axis=0))

In [102]:
parseSentenceRemovePunc((1,"I'm so pumped!!! for the runn* lol yeet., hi, there"))

['im', 'so', 'pumped', 'for', 'the', 'runn', 'lol', 'yeet', 'hi', 'there']

In [ ]:
corpusRP = parseSentences(sentences, parseSentenceRemovePunc)
word2VecRP = Word2Vec(corpus, size=256, min_count=1)
corpusVecsRP = [reviewToVecRemovePunc(s) for s in corpusRP]